# Importing Dependencies

In [1]:
import soundfile
import numpy as np
import librosa
import glob
import os
import numpy as np
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow import keras
from keras import optimizers
%load_ext tensorboard
%matplotlib inline
from sklearn.model_selection import train_test_split

#%reload_ext tensorboard

# Feature Extraction and Loading Dataset

In [2]:
def extract_feature(file_name, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [3]:
def load_data(train_size=0.8,test_size=0.2):
    X, y = [], []
    try :
        for file in glob.glob("C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset/Actor_*/*.wav"):
            # get the base name of the audio file
            print(file)
            basename = os.path.basename(file)
            print(basename)
          # get the emotion label
            emotion = int2emotion[basename.split("-")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
            if emotion not in AVAILABLE_EMOTIONS:
                continue
          # extract speech features
            features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
            X.append(features)
            l={'happy':0.0,'sad':1.0,'neutral':3.0,'angry':4.0}
            y.append(l[emotion])
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size,train_size=train_size,random_state=7)

# Defining Emotions

In [4]:
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}


In [5]:
X_train, X_test, y_train, y_test = load_data()

C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-01-01-01-01-01.wav
03-01-01-01-01-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-01-01-01-02-01.wav
03-01-01-01-01-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-01-01-02-01-01.wav
03-01-01-01-02-01-01.wav


C:\Users\hp\AppData\Local\Temp\ipykernel_16348\90870015.py:20: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\hp\AppData\Local\Temp\ipykernel_16348\90870015.py:20: FutureWarning: Pass y=[ 3.0517578e-05  3.0517578e-05  3.0517578e-05 ...  0.0000000e+00
  0.0000000e+00 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\hp\AppData\Local\Temp\ipykernel_16348\90870015.py:20: FutureWarning: Pass y=[3.0517578e-05 3.0517578e-05 0.0000000e+00 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-01-01-02-02-01.wav
03-01-01-01-02-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-01-01-01-01.wav
03-01-02-01-01-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-01-01-02-01.wav
03-01-02-01-01-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-01-02-01-01.wav
03-01-02-01-02-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-01-02-02-01.wav
03-01-02-01-02-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-02-01-01-01.wav
03-01-02-02-01-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-02-01-02-01.wav
03-01-02-02-01-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-02-02-02-01-01.wav
03-01-02-02-02-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-

C:\Users\hp\AppData\Local\Temp\ipykernel_16348\90870015.py:20: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.0517578e-05 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\hp\AppData\Local\Temp\ipykernel_16348\90870015.py:20: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 3.0517578e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-03-02-02-01-01.wav
03-01-03-02-02-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-03-02-02-02-01.wav
03-01-03-02-02-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-04-01-01-01-01.wav
03-01-04-01-01-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-04-01-01-02-01.wav
03-01-04-01-01-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-04-01-02-01-01.wav
03-01-04-01-02-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-04-01-02-02-01.wav
03-01-04-01-02-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-04-02-01-01-01.wav
03-01-04-02-01-01-01.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_01\03-01-04-02-01-02-01.wav
03-01-04-02-01-02-01.wav
C:/Users/hp/MINI_PROJECT-Speech-

...

C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-01-01-02-02-23.wav
03-01-01-01-02-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-02-01-01-01-23.wav
03-01-02-01-01-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-02-01-01-02-23.wav
03-01-02-01-01-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-02-01-02-01-23.wav
03-01-02-01-02-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-02-01-02-02-23.wav
03-01-02-01-02-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-02-02-01-01-23.wav
03-01-02-02-01-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-02-02-01-02-23.wav
03-01-02-02-01-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-02-02-02-01-23.wav
03-01-02-02-02-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-

C:\Users\hp\AppData\Local\Temp\ipykernel_16348\90870015.py:20: FutureWarning: Pass y=[-3.0517578e-05 -3.0517578e-05  0.0000000e+00 ...  3.0517578e-05
  3.0517578e-05  3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


03-01-06-01-01-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-06-01-01-02-23.wav
03-01-06-01-01-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-06-01-02-01-23.wav
03-01-06-01-02-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-06-01-02-02-23.wav
03-01-06-01-02-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-06-02-01-01-23.wav
03-01-06-02-01-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-06-02-01-02-23.wav
03-01-06-02-01-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-06-02-02-01-23.wav
03-01-06-02-02-01-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-06-02-02-02-23.wav
03-01-06-02-02-02-23.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_23\03-01-07-01-01-01-23.wav
03-01-07-01-01-01-23.wav
C:/User

C:\Users\hp\AppData\Local\Temp\ipykernel_16348\90870015.py:20: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.5258789e-04
 -1.5258789e-04 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


03-01-05-01-01-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-01-02-01-24.wav
03-01-05-01-02-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-01-02-02-24.wav
03-01-05-01-02-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-02-01-01-24.wav
03-01-05-02-01-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-02-01-02-24.wav
03-01-05-02-01-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-02-02-01-24.wav
03-01-05-02-02-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-05-02-02-02-24.wav
03-01-05-02-02-02-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-06-01-01-01-24.wav
03-01-06-01-01-01-24.wav
C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset\Actor_24\03-01-06-01-01-02-24.wav
03-01-06-01-01-02-24.wav
C:/User

In [6]:
X_train = np.asarray(X_train)
y_train= np.asarray(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)
X_train.shape,y_train.shape,X_test.shape,y_test.shape
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)
x_traincnn.shape,x_testcnn.shape

((537, 180, 1), (135, 180, 1))

# Configuration 1 - CNN_Adam

In [7]:
import tensorflow as tf
model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))        #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))



model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(8))                                                 #3
model.add(Activation('softmax'))
optimizer=keras.optimizers.Adam(lr=0.001)
model.summary()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 180, 128)          768       
                                                                 
 activation (Activation)     (None, 180, 128)          0         
                                                                 
 dropout (Dropout)           (None, 180, 128)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 22, 128)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_1 (Activation)   (None, 22, 128)           0         
                                                        

C:\Users\hp\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
import datetime
import tensorflow
%reload_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir logs/fit



In [9]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=20, epochs=50, validation_data=(x_testcnn, y_test),callbacks=[tensorboard_callback])

Epoch 1/50
27/27 [==============================] - 4s 67ms/step - loss: 4.1007 - accuracy: 0.3073 - val_loss: 1.9732 - val_accuracy: 0.4667
Epoch 2/50
27/27 [==============================] - 1s 33ms/step - loss: 2.6557 - accuracy: 0.3408 - val_loss: 2.1168 - val_accuracy: 0.3481
Epoch 3/50
27/27 [==============================] - 1s 32ms/step - loss: 1.6290 - accuracy: 0.4302 - val_loss: 1.1655 - val_accuracy: 0.4963
Epoch 4/50
27/27 [==============================] - 1s 30ms/step - loss: 1.4329 - accuracy: 0.4246 - val_loss: 1.1259 - val_accuracy: 0.5185
Epoch 5/50
27/27 [==============================] - 1s 31ms/step - loss: 1.2434 - accuracy: 0.4562 - val_loss: 1.0140 - val_accuracy: 0.5259
Epoch 6/50
27/27 [==============================] - 1s 32ms/step - loss: 1.2074 - accuracy: 0.4842 - val_loss: 0.9723 - val_accuracy: 0.5704
Epoch 7/50
27/27 [==============================] - 1s 33ms/step - loss: 1.0925 - accuracy: 0.4786 - val_loss: 0.8964 - val_accuracy: 0.5852
Epoch 8/50
27

In [10]:
em=['happy','sad','neutral','angry']

# Predicting using Config-1

In [11]:
predictions = model.predict(x_testcnn)
n=predictions[1]
print(em[1])
print(n)

5/5 [==============================] - 0s 8ms/step
sad
[5.91337346e-02 1.51712456e-07 1.99179841e-21 3.66753484e-05
 9.40829515e-01 9.27262681e-21 1.01231055e-19 1.24379479e-21]


In [12]:
loss, acc = model.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5/5 [==============================] - 0s 10ms/step - loss: 0.7738 - accuracy: 0.6889
Restored model, accuracy: 68.89%


In [13]:
filename = "C:/Users/hp/MINI_PROJECT-Speech-Emotion-Recognization/dataset/Actor_02/03-01-01-01-02-01-02.wav"
features = np.array(extract_feature(filename, mfcc=True, chroma=True, mel=True).reshape(1, -1))
f=np.expand_dims(features,axis=2)
result = model.predict(f)[0]
print("result :",em[int(result[0])])

1/1 [==============================] - 0s 39ms/step
result : happy


C:\Users\hp\AppData\Local\Temp\ipykernel_16348\90870015.py:20: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00 0.0000000e+00
 3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


# Configuration 2 - CNN_RMSProp

In [14]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

from tensorflow import keras
from keras import optimizers


um = Sequential()

um.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))#1
um.add(Activation('relu'))
um.add(Dropout(0.25))
um.add(MaxPooling1D(pool_size=(8)))

um.add(Conv1D(128, 5,padding='same',))                  #2
um.add(Activation('relu'))
um.add(MaxPooling1D(pool_size=(8)))
um.add(Dropout(0.25))

um.add(Conv1D(128, 5,padding='same',))                  #3
um.add(Activation('relu'))
um.add(Dropout(0.25))

um.add(Flatten())
um.add(Dense(8))                                        #4                      
um.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(learning_rate=0.00005,epsilon=None,rho=0.9,decay=0.0)

um.summary()

um.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 180, 128)          768       
                                                                 
 activation_3 (Activation)   (None, 180, 128)          0         
                                                                 
 dropout_2 (Dropout)         (None, 180, 128)          0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 22, 128)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_4 (Activation)   (None, 22, 128)           0         
                                                      

In [15]:
umhistory=um.fit(x_traincnn, y_train, batch_size=20, epochs=50, validation_data=(x_testcnn, y_test),callbacks=[tensorboard_callback])


Epoch 1/50
27/27 [==============================] - 4s 58ms/step - loss: 13.4428 - accuracy: 0.1210 - val_loss: 2.3872 - val_accuracy: 0.3333
Epoch 2/50
27/27 [==============================] - 1s 34ms/step - loss: 7.6372 - accuracy: 0.2421 - val_loss: 2.6835 - val_accuracy: 0.3630
Epoch 3/50
27/27 [==============================] - 1s 32ms/step - loss: 6.6623 - accuracy: 0.2737 - val_loss: 2.2811 - val_accuracy: 0.3704
Epoch 4/50
27/27 [==============================] - 1s 30ms/step - loss: 6.0542 - accuracy: 0.2905 - val_loss: 2.1519 - val_accuracy: 0.4000
Epoch 5/50
27/27 [==============================] - 1s 33ms/step - loss: 5.2948 - accuracy: 0.3333 - val_loss: 2.4952 - val_accuracy: 0.4000
Epoch 6/50
27/27 [==============================] - 1s 29ms/step - loss: 5.4085 - accuracy: 0.2868 - val_loss: 1.7126 - val_accuracy: 0.4296
Epoch 7/50
27/27 [==============================] - 1s 31ms/step - loss: 4.9976 - accuracy: 0.2644 - val_loss: 1.6750 - val_accuracy: 0.4370
Epoch 8/50
2

In [16]:
%reload_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16380), started 0:01:37 ago. (Use '!kill 16380' to kill it.)

# Predicting using Config - 2

In [17]:
loss, acc = um.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5/5 [==============================] - 0s 10ms/step - loss: 1.2153 - accuracy: 0.5037
Restored model, accuracy: 50.37%


# Configuration 2 - CNN2_Adam

In [18]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow import keras
from keras import optimizers


sm = Sequential()

sm.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))#1
sm.add(Activation('relu'))
sm.add(Dropout(0.1))
sm.add(MaxPooling1D(pool_size=(8)))

sm.add(Conv1D(128, 5,padding='same',))                  #2
sm.add(Activation('relu'))
sm.add(MaxPooling1D(pool_size=(8)))
sm.add(Dropout(0.1))

sm.add(Conv1D(128, 5,padding='same',))                  #3
sm.add(Activation('relu'))
sm.add(Dropout(0.1))

sm.add(Conv1D(128, 5,padding='same',))                  #4
sm.add(Activation('relu'))
sm.add(Dropout(0.1))

sm.add(Flatten())
sm.add(Dense(8))                                        #5                     
sm.add(Activation('softmax'))

sm.summary()
sm.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 180, 128)          768       
                                                                 
 activation_7 (Activation)   (None, 180, 128)          0         
                                                                 
 dropout_5 (Dropout)         (None, 180, 128)          0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 22, 128)          0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_8 (Activation)   (None, 22, 128)           0         
                                                      

In [19]:
smhistory=sm.fit(x_traincnn, y_train, batch_size=20, epochs=50, validation_data=(x_testcnn, y_test),callbacks=[tensorboard_callback])

Epoch 1/50
27/27 [==============================] - 4s 51ms/step - loss: 2.7081 - accuracy: 0.2812 - val_loss: 1.3347 - val_accuracy: 0.4370
Epoch 2/50
27/27 [==============================] - 1s 32ms/step - loss: 1.4238 - accuracy: 0.3333 - val_loss: 1.2959 - val_accuracy: 0.3556
Epoch 3/50
27/27 [==============================] - 1s 30ms/step - loss: 1.3077 - accuracy: 0.3911 - val_loss: 1.3061 - val_accuracy: 0.4296
Epoch 4/50
27/27 [==============================] - 1s 31ms/step - loss: 1.3154 - accuracy: 0.3892 - val_loss: 1.1946 - val_accuracy: 0.4074
Epoch 5/50
27/27 [==============================] - 1s 35ms/step - loss: 1.2312 - accuracy: 0.4339 - val_loss: 1.1054 - val_accuracy: 0.5333
Epoch 6/50
27/27 [==============================] - 1s 32ms/step - loss: 1.2415 - accuracy: 0.4227 - val_loss: 1.1284 - val_accuracy: 0.5481
Epoch 7/50
27/27 [==============================] - 1s 33ms/step - loss: 1.2345 - accuracy: 0.4264 - val_loss: 1.1508 - val_accuracy: 0.4074
Epoch 8/50
27

In [20]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16380), started 0:02:25 ago. (Use '!kill 16380' to kill it.)

# Predicting using Config - 3

In [21]:
loss, acc = sm.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5/5 [==============================] - 0s 9ms/step - loss: 0.7018 - accuracy: 0.6667
Restored model, accuracy: 66.67%


# Configuration 4 - LSTM_RMSProp

In [22]:
from keras.models import Sequential
from keras import layers
from keras import optimizers
from keras import callbacks

In [23]:
model_LSTM=Sequential()
model_LSTM.add(layers.LSTM(64,return_sequences=True,input_shape=(180,1)))
model_LSTM.add(layers.LSTM(64))
model_LSTM.add(layers.Dense(8,activation='softmax'))
print(model_LSTM.summary())
model_LSTM.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 180, 64)           16896     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 8)                 520       
                                                                 
Total params: 50,440
Trainable params: 50,440
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
umhistory=model_LSTM.fit(x_traincnn, y_train, batch_size=20, epochs=50, validation_data=(x_testcnn, y_test),callbacks=[tensorboard_callback])

Epoch 1/50
27/27 [==============================] - 19s 261ms/step - loss: 1.4973 - accuracy: 0.2607 - val_loss: 1.4294 - val_accuracy: 0.2296
Epoch 2/50
27/27 [==============================] - 5s 179ms/step - loss: 1.3785 - accuracy: 0.2756 - val_loss: 1.3444 - val_accuracy: 0.3407
Epoch 3/50
27/27 [==============================] - 5s 189ms/step - loss: 1.3631 - accuracy: 0.3203 - val_loss: 1.3756 - val_accuracy: 0.3037
Epoch 4/50
27/27 [==============================] - 5s 188ms/step - loss: 1.3718 - accuracy: 0.2756 - val_loss: 1.3530 - val_accuracy: 0.2296
Epoch 5/50
27/27 [==============================] - 6s 210ms/step - loss: 1.3716 - accuracy: 0.2551 - val_loss: 1.3598 - val_accuracy: 0.2296
Epoch 6/50
27/27 [==============================] - 5s 173ms/step - loss: 1.3658 - accuracy: 0.2793 - val_loss: 1.3658 - val_accuracy: 0.2296
Epoch 7/50
27/27 [==============================] - 5s 171ms/step - loss: 1.3659 - accuracy: 0.2849 - val_loss: 1.3781 - val_accuracy: 0.2296
Epoch

# Evaluating Config - 4 LSTM

In [25]:
loss, acc = model_LSTM.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5/5 [==============================] - 0s 72ms/step - loss: 0.9576 - accuracy: 0.5407
Restored model, accuracy: 54.07%


# Performance Analysis and Visualization of all Configurations

In [26]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16380), started 0:06:48 ago. (Use '!kill 16380' to kill it.)